# データ取得(NVIDIA)

# Import

In [1]:
import yfinance as yf
import pandas as pd
import sys
sys.path.append("./src")

In [2]:
import PreProcess
import Learn
import DataSets
import Models

In [ ]:
#run if needed
# NVIDIAのティッカーシンボル（NVDA）
ticker = "NVDA"
# 20年分のデータを取得
data = yf.download(ticker, start="2004-01-01", end="2024-01-01")
# データを確認
print(data.head())

In [ ]:
filename = "nvidia_20y_stock_data.csv"
data.to_csv(f"./RawData/{filename}")

In [ ]:
filename = "nvidia_20y_stock_data.csv"
data = pd.read_csv(f"./RawData/{filename}")
print(data.columns)

# PreProcess

In [3]:
#load csv file
input_path = "./RawData/nvidia"
output_path = "./PreProcessedData/nvidia"
train_rate = 0.8
eval_rate = 0.1
test_rate = 0.1
frames_num = 15 # LSTMならseq_lenと等価
pred_future_frame = 1 #次の日 or 3日後 (恐らく土日を挟んでいる)
target_cols =  ['Adj Close', 'Close',  'High', 'Low',  'Open', 'Volume']

PreProcess = PreProcess.PreProcess(train_rate = train_rate, eval_rate = eval_rate, test_rate = test_rate)
PreProcess.preprocess_edit(input_path = input_path, output_path = output_path, frames_num = frames_num, pred_future_frame = pred_future_frame, target_cols = target_cols)

100%|██████████| 1/1 [00:06<00:00,  6.41s/it]


In [4]:
#debug
import torch
file = torch.load("PreProcessedData/nvidia/Train/X/4046.pt")
print(file.shape)

torch.Size([15, 6])


C:\Users\hidak\AppData\Local\Temp\ipykernel_8892\856376336.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  file = torch.load("PreProcessedData/nvidia/Train/X/4046.pt")


# Learn

In [1]:
import yfinance as yf
import pandas as pd
import sys
sys.path.append("./src")
import PreProcess
import Learn
import DataSets
import Models

In [ ]:
points_num = 6
frames_num = 15
pred_future_frame = 1
batch_size = 10
modelSaveSpan = 10
seq_len = 15
folder_path = "PreProcessedData/nvidia"
learn = Learn.Learn(points_num = points_num, frames_num = frames_num, pred_future_frame = pred_future_frame, batch_size = batch_size, seq_len = seq_len, modelSaveSpan = modelSaveSpan)
loader = learn.data_loader_forLSTM(folder_path = folder_path, DataSetsClass = DataSets.StockPriceDataSet_forLSTM)
criterion, optimizer = learn.network_forLSTM(learning_Model = Models.StockPriceLSTM(input_size = points_num, hidden_size = 32, num_layers = 3, dropout = 0.1, seq_len = 15, output_size = 6), lr = 1e-4, weight_decay = 5e-5)
learn.learn_forLSTM(criterion=criterion, optimizer=optimizer, n_epochs = 2500)

tensorboard --logdir=runs/Aug_CF_1_20241110181706


  0%|          | 0/4047 [00:00<?, ?it/s]c:\Users\hidak\research\StockPrediction\src\DataSets.py:189: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.dataX.append(torch.lo

torch.Size([15, 6]) torch.Size([6])


  1%|          | 26/2500 [00:51<1:23:13,  2.02s/it]